In [3]:
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T
from torchvision import transforms
from models.erfnet_road import ERFNet
import cv2
from inference import preprocess, segmentation, visualize

In [4]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
np.random.seed(50)
torch.manual_seed(50)

if torch.cuda.is_available():
    torch.cuda.manual_seed(50)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
net = ERFNet(num_classes=4)

In [6]:
checkpoint = torch.load('./pretrained_models/ERF_epoch_32_acc_0.7833.pt',map_location=device)
multigpus = True
for key in checkpoint:  # check if the model was trained in multiple gpus
    if 'module' in key:
        multigpus = multigpus and True
    else:
        multigpus = False
if multigpus:
    net = torch.nn.DataParallel(net)
net.load_state_dict(checkpoint)


<All keys matched successfully>

In [7]:
input_path = 'freespace_lanemarking_segmentation_data_test/img_test/autopilot_test_0001_20101230_021815_000172.png'
img = Image.open(input_path)
origin_height = img.size[1]
origin_width = img.size[0]
preprocess_img = preprocess(img, height=360, width=640)

seg_map = segmentation(preprocess_img, origin_height, origin_width, net, device)

overlaid_img = visualize(seg_map, np.asarray(img))

combined_img = np.concatenate((np.asarray(img), overlaid_img), axis=1)

cv2.imwrite('ouput.jpg', combined_img)


True